In [ ]:
import numpy as np
np.printoptions(suppress= True)

In [ ]:
def kaczmarz(A, b, alpha= 1, TOL= 1e-16):
    """
    Implement the Kaczmarz algorithm to solve a system of linear equation.

    The algorithm works under the principle of finding an estimated solution (xhat) that satisfies 
    ||A@xhat - b||^2 < e, with e is the tolerance value.
    """
    m, n = A.shape
    
    # Initialize the initial solution
    X = np.zeros(n)
    
    k = 0
    while True:
        # Sampling the i-th row of A and i-th coordinate for b
        i = k % m
        ai = A[i,:]
        bi = b[i]

        # Find the solution at the k+1 iteration
        Xnew = X + alpha * ((bi - np.dot(ai,X.T)) / np.linalg.norm(ai) ** 2 * ai)
        
        # Compute the error
        err = np.linalg.norm(np.dot(A,Xnew.T) - b) ** 2
        if err < TOL:
            break
        X = Xnew
        k += 1
    return X

def admm(d, A, b, rho= 1, max_iter= 5):
    """
    Implement the alternating direction method of multipliers (ADMM) optimization algorithm to solve a linear program.
    The implementation is based on the formulas mentioned in Liu et al. (2018).

    The linear program has the following form: min d^Tx, s.t. Ax = b
    """
    d = d.T
    b = b.T

    # Extract the number of variables and constraints from the matrix dimensions
    no_of_vars = A.shape[1]
    no_of_constraints = A.shape[0]

    # Initialize the dual variables y and mu
    y  = np.random.normal(size= (1,no_of_vars))
    mu = np.random.normal(size= (1,no_of_vars))

    # Construct the augmented Lagrangian matrix
    C = np.block([
        [rho * np.eye(no_of_vars), A.T],
        [A, np.zeros((no_of_constraints,no_of_constraints))]
                ])

    for i in range(max_iter):
        # Solve for the primal variable x
        alpha = y - (1/rho) * (mu + d)
        m = np.block([rho * alpha, b]).T
        x = kaczmarz(C, m)[: no_of_vars]

        # Apply soft-thresholding to update the dual variable y
        beta = x + (1/rho) * mu.T
        beta[beta < 0] = 0
        y = beta.T

        # Update the dual variable mu
        mu = mu + rho * (x.T - y)

    return (x,y,mu)

In [3]:
d = np.array([-1, -2, -4, 0, 0, 0])
A = np.array([
    [3, 1, 5, 1, 0, 0],
    [1, 4, 1, 0, 1, 0],
    [2, 0, 2, 0, 0, 1]])
b = np.array([10, 8, 7])

x, _ , _ = admm(d, A, b)
print(x)